<a href="https://colab.research.google.com/github/UDulius/NLP/blob/main/PotentialCourseWorkFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [225]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',  categories=categories, shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test',  categories=categories, shuffle=True, random_state=42)

In [226]:
index = input('type your student number')

type your student number29008326


In [227]:
x=divmod(int(index),4)
yourdata1=x[1]
y=divmod(int(index),3)
yourdata2=y[1]

print('This is your data set index ----> (', x[1], y[1], ')')

This is your data set index ----> ( 2 0 )


In [228]:
data1= twenty_train.target_names[x[1]]
data2= twenty_train.target_names[y[1]]
categories1=[data1,data2]
print(categories1)

['sci.med', 'alt.atheism']


In [229]:
print(data1)

sci.med


In [350]:
import nltk
import re
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
from tqdm import tqdm
from nltk.corpus import stopwords
englishStopwords = stopwords.words('english')
from nltk.corpus import wordnet
nltk.download('wordnet')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [355]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
senti = SentimentIntensityAnalyzer()


def findLemma(word):
  lemma = wordnet.morphy(word)
  if lemma is not None:
    return lemma
  else:
    return word

def getTags(text):
  token = word_tokenize(text)
  token = [l.lower() for l in token]
  train_tags = nltk.pos_tag(token)
  return [i[1] for i in train_tags]

def processText(text, lemma=False, gram=1, rmStop=True):
  text = re.sub(r'(http|https)?:\/\/(\w|/.|\/|\?|\=|\%|\%)*\b|@\w+|#', '', text, flags=re.MULTILINE) #Deletes any URLs, hashtags, and @text
  tokens = word_tokenize(text) 
  whitelist = ["n't", "not", "no"]
  new_tokens = []
  stoplist = englishStopwords if rmStop else []
  for i in tokens:
    i = i.lower()
    if i.isalpha() and (i not in stoplist or i in whitelist):
      if lemma: i = findLemma(i)
      new_tokens.append(i)
  del tokens

  if gram<=1:
    return new_tokens
  else:
    return [' '.join(i) for i in nltk.ngrams(new_tokens, gram)]


In [356]:
twenty_train1 = fetch_20newsgroups(subset='train',  categories=categories1, shuffle=True, random_state=42)
twenty_test1 = fetch_20newsgroups(subset='test',  categories=categories1, shuffle=True, random_state=42)

In [357]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression

text_clf = Pipeline([
    ('vect', CountVectorizer(analyzer=processText, max_df=20, min_df=4)), 
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', LogisticRegression())
])

In [358]:
text_clf.fit(twenty_train1.data, twenty_train1.target)

AttributeError: ignored

In [323]:
predicted = text_clf.predict(twenty_test1.data)

In [324]:
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(twenty_test1.target, predicted))

print(metrics.classification_report(twenty_test1.target, predicted, target_names=twenty_test1.target_names))

pd.DataFrame(metrics.confusion_matrix(twenty_test1.target, predicted),
             columns=twenty_test1.target_names,index=twenty_test1.target_names)

Accuracy: 0.9202797202797203
              precision    recall  f1-score   support

 alt.atheism       0.99      0.83      0.90       319
     sci.med       0.88      0.99      0.93       396

    accuracy                           0.92       715
   macro avg       0.93      0.91      0.92       715
weighted avg       0.93      0.92      0.92       715



,alt.atheism,sci.med
alt.atheism,266,53
sci.med,4,392


In [325]:
df_pred = pd.DataFrame({'news':twenty_test1.data,'prediction':predicted, 'true':twenty_test1.target})
df_pred[df_pred['true'] != df_pred['prediction']]

,news,prediction,true
21,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,1,0
30,From: darice@yoyo.cc.monash.edu.au (Fred Rice)...,1,0
31,From: Nanci Ann Miller <nm0w+@andrew.cmu.edu>\...,1,0
35,From: kax@cs.nott.ac.uk (Kevin Anthoney)\nSubj...,1,0
57,From: aaron@minster.york.ac.uk\nSubject: Re: G...,1,0
66,From: nyeda@cnsvax.uwec.edu (David Nye)\nSubje...,1,0
106,From: mccullou@snake10.cs.wisc.edu (Mark McCul...,1,0
114,From: mayne@pipe.cs.fsu.edu (William Mayne)\nS...,1,0
115,"From: ""Robert Knowles"" <p00261@psilink.com>\nS...",1,0
124,From: jgreen@trumpet.calpoly.edu (James Thomas...,1,0
